# This Notebook contains utilities for 

1. Generating Test Id's for users
2. Performing some analytics on the data

# Step 0: Provide Firebase Admin SDK key + Database URL

Follow instructions here to get servicekey.json: https://firebase.google.com/docs/admin/setup#python:~:text=To%20generate%20a%20private%20key%20file%20for%20your%20service%20account%3A

Then upload that .json file to colab in the base directory

The code will also ask for the realtime database URL which can be found in the realtime database firebase console:


In [1]:
# run this cell to initialize firebase
! pip install -q firebase_admin

# get firebase.json file functionality
import os

def get_first_json_file() -> str:
  current_directory = os.getcwd()  # Get the current directory
  files = os.listdir(current_directory)  # Get all files in the directory

  # Filter the files to keep only the ones ending with ".json"
  json_files = [file for file in files if file.endswith(".json")]

  if json_files:
    return json_files[0]  # Return the first JSON file name
  else:
    return None

# check if json file exists
if not get_first_json_file():
  print("No JSON files found in the current directory.")
  raise

# get database URL
databaseURL = input('Enter the realtime database URL: ')

# initialize firebase
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate(get_first_json_file())
default_app = firebase_admin.initialize_app(cred, {
    'databaseURL': databaseURL
})

Enter the realtime database URL: https://drt2-62447-default-rtdb.firebaseio.com/


## 1. Generating Test Id's for users

In [2]:
import string
import random
# Function to generate a test id

def generate_test_id(name="") -> str:
  name = name.replace(" ", "")
  test_id: str = name
  if len(name) < 5:
    test_id += ''.join([random.choice(string.ascii_letters) for _ in range(5 - len(name))])
  test_id += ''.join([random.choice(string.digits) for _ in range(5)])

  return test_id

In [3]:
generate_test_id()

'lsKbL45440'

In [7]:
# Import database module.
from firebase_admin import db

# Get a database reference to our blog.
ref = db.reference('Tests Taken')

# Generate and add the test id
def add_test_id_to_database(name="") -> str:
  test_id = generate_test_id(name)
  ref.update({
      test_id: {
          "0" : "Patient comes to the clinic complaining of chest pain"
      }
  })
  return test_id

### Main function to add a test_id to the database

In [5]:
add_test_id_to_database()

# 2. Performing analytics on the data

This code can be used to retrieve data from firebase and analyze the resulting data. Currently it only analyzes on a case by case basis, but should be easily extensible to whatever metrics are needed

In [8]:
# Import database module.
from firebase_admin import db

# Get a database reference to our blog.
ref = db.reference('Tests Taken')

all_test_data = ref.get()
all_test_data

{'chest': ['Patient comes to the clinic complaining of chest pain',
  {'CaseStartTime': '1682305832365',
   'Clicks': {'1682305837907': 'Chest pain lasting 1-3 hours',
    '1682305838426': 'Chest pain lasting a few seconds',
    '1682305838960': 'Chest pain lasting greater than 24 hours',
    '1682305840343': 'Chest pain burning',
    '1682305840973': 'Chest pain dull\\pressure\\squeezing',
    '1682305856427': 'Location of chest pain posterior thoracic',
    '1682305856825': 'Location of chest pain lateral to costochondral junction',
    '1682305857236': 'Location of chest pain at sternochondral or costochondral junction',
    '1682305857805': 'Location of chest pain substernal and\\pr left precordial',
    '1682305858204': 'Radiation of chest pain to back',
    '1682305858779': 'Radiation of chest pain to neck\\jaw\\arm'},
   'FinalDiagnosis': {'Confidence': 'Somewhat',
    'Primary': 'Thoracic Aortic Aneurysm',
    'Secondary': 'Pneumonia',
    'Tertiary': 'Upper GI',
    'Timestamp

In [19]:
import datetime
def date_to_string(unix_timestamp: int) -> str:
  dt = datetime.datetime.fromtimestamp(unix_timestamp / 1000)

  # Extract minutes and seconds from the datetime object
  minutes = dt.minute
  seconds = dt.second
  milliseconds = dt.microsecond // 1000

  # Format the minutes and seconds as MM:SS.MM
  return "{:02d}:{:02d}.{:02d}".format(minutes, seconds, milliseconds)

for test_id, case_list in all_test_data.items():
  # only look at completed cases
  if len(case_list) <= 2:
    continue
  
  answer = ["", "Myocardial Infarction", "Pulmonary Embolism"]

  # print out analytics for this completed case
  for case_num in [1, 2]:
    case_data = case_list[case_num]

    features_order_list = [feature for timestamp, feature in case_data['Clicks'].items()]
    time_for_case = int(case_data['FinalDiagnosis']['Timestamp']) - int(case_data['CaseStartTime'])
    time_per_feature = time_for_case / len(features_order_list)
    diag_top3_working = answer[case_num] in case_data['WorkingDiagnosis'].values()
    diag_top1_working = answer[case_num] == case_data['WorkingDiagnosis']['Primary']
    diag_top3_submit = answer[case_num] in case_data['FinalDiagnosis'].values()
    diag_top1_submit = answer[case_num] == case_data['FinalDiagnosis']['Primary']

    print(f"Test id: {test_id}")
    print("Features order: ", features_order_list)
    print(f"Total time for case: {date_to_string(time_for_case)}")
    print(f"Time per feature: {date_to_string(time_per_feature)}")
    print(f"Was diagnosis in top 3 at the working differential stage? {diag_top3_working}")
    print(f"Was diagnosis in top 1 at the working differential stage? {diag_top1_working}")
    print(f"Was diagnosis in top 3 at final submit stage? {diag_top3_submit}")
    print(f"Was diagnosis in top 1 at final submit stage? {diag_top1_submit}")
    print()

Test id: chest
Features order:  ['Chest pain lasting 1-3 hours', 'Chest pain lasting a few seconds', 'Chest pain lasting greater than 24 hours', 'Chest pain burning', 'Chest pain dull\\pressure\\squeezing', 'Location of chest pain posterior thoracic', 'Location of chest pain lateral to costochondral junction', 'Location of chest pain at sternochondral or costochondral junction', 'Location of chest pain substernal and\\pr left precordial', 'Radiation of chest pain to back', 'Radiation of chest pain to neck\\jaw\\arm']
Total time for case: 00:30.699
Time per feature: 00:02.790
Was diagnosis in top 3 at the working differential stage? False
Was diagnosis in top 1 at the working differential stage? False
Was diagnosis in top 3 at final submit stage? False
Was diagnosis in top 1 at final submit stage? False

Test id: chest
Features order:  ['Chest pain lasting 1-3 hours', 'Chest pain lasting a few seconds', 'Chest pain lasting greater than 24 hours', 'Chest pain burning', 'Chest pain dull\\